In [ ]:
import os
import sys
sys.path.insert(0, "../..")
sys.path.insert(0, "../../../acquire")
import ipywidgets as widgets

from HUGS.Processing import search

from Acquire.Client import User, Drive, Service, PAR, Authorisation, StorageCreds

## Start with uploading and processing of the data

In [ ]:
base_url = "https://hugs.acquire-aaai.com/t"
hugs_url = "https://hugs.acquire-aaai.com"
user = User(username="gareth", identity_url="%s/identity" % base_url)
response = user.request_login()

Login by visiting: https://login.acquire-aaai.com?id=a0-a6/b1.18.e5.6a
(please check that this page displays the message 'small cows sit quickly')


Check we're logged in

In [ ]:
user.wait_for_login()

True

### Upload a file for processing


In [ ]:
filename = "bsd.picarro.1minute.248m.dat"
dir_path = os.path.abspath("")
test_data = "../../test/data/proc_test_data/CRDS"
filepath = os.path.join(dir_path, test_data, filename)

In [ ]:
creds = StorageCreds(user=user, service_url="%s/storage" % base_url)
drive = Drive(creds=creds, name="test_drive")
filemeta = drive.upload(filepath)
par = PAR(location=filemeta.location(), user=user)

In [ ]:
hugs = Service(service_url="%s/hugs" % base_url)
par_secret = hugs.encrypt_data(par.secret())
auth = Authorisation(resource="process", user=user)

Clear the Datasources from CRDS and GC objects (currently Datasource UUIDs are just being randomly generated and we get multiple Datasources holding the same data)

In [ ]:
response = hugs.call_function(function="clear_datasources", args={})

In [ ]:
args = {"authorisation": auth.to_data(),
            "par": {"data": par.to_data()},
            "par_secret": {"data": par_secret},
            "data_type": "CRDS"}

response = hugs.call_function(function="process", args=args)

In [ ]:
search_results = None

In [ ]:
from HUGS.Processing import search

search_terms = widgets.Text(
                    value="Search",
                    placeholder = "Search",
                    description = "Search terms:",
                    disabled = False
)

locations = widgets.Text(
                value="Locations",
                placeholder = "BSD, HFD",
                description = "Locations:",
                disabled = False
)

data_type = widgets.Dropdown(
                options=["CRDS", "GC"], 
                value="CRDS",
                description="Data type",
                disabled=False
    )



layout=widgets.Layout(display = "flex", width = "50%")
search_button = widgets.Button(description="Search", button_style="success", \
                               layout=widgets.Layout(flex='1 1 0%', width='25%'))

start_picker = widgets.DatePicker(
                    description='Start date',
                    disabled=False
)

end_picker = widgets.DatePicker(
                    description='End date',
                    disabled=False
)


box = widgets.VBox(children=[search_terms, locations, start_picker, end_picker, data_type, search_button], layout=layout)



def call_search(x):
    """ Call the search function and pass it the values 
        in the text boxes
            
    """
    from datetime import datetime
    from Acquire.ObjectStore import datetime_to_string
    start = datetime.combine(start_picker.value, datetime.min.time())
    end = datetime.combine(end_picker.value, datetime.min.time())
    
    args = {}
    args["search_terms"] = search_terms.value
    
    # Remove whitespace and commas from the search terms
    split_search_terms = search_terms.value.replace(" ", "").split(",")
    args["search_terms"] = split_search_terms

    split_locations = locations.value.replace(" ", "").split(",")
    args["locations"] = split_locations
      
    args["locations"] = locations.value
    args["data_type"] = data_type.value
    args["start_datetime"] = datetime_to_string(start)
    args["end_datetime"] = datetime_to_string(end)
    
    global search_results
    search_results = hugs.call_function(function="search", args=args)
    
    print(search_results)
    
search_button.on_click(call_search)



In [ ]:
start_picker.value

In [ ]:
box

{'results': {'bsd_co': ['data/uuid/053d1593-6023-40ef-9912-168bad7cf50d/2014-01-30T10:52:30_2014-01-30T14:20:30'], 'bsd_co2': ['data/uuid/7f69fe31-1094-4227-b556-402f2e3c99c5/2014-01-30T10:52:30_2014-01-30T14:20:30']}}


In [ ]:
results = search_results["results"]

from collections import defaultdict

keys = defaultdict(list)
for x in results.keys():
    for key in results[x]:
        keys[x].append(key)
        
data_args = {"return_type":"json", "keys": keys}



data = hugs.call_function(function="retrieve", args=data_args)

In [ ]:
print(keys)

defaultdict(<class 'list'>, {'bsd_co': ['data/uuid/053d1593-6023-40ef-9912-168bad7cf50d/2014-01-30T10:52:30_2014-01-30T14:20:30'], 'bsd_co2': ['data/uuid/7f69fe31-1094-4227-b556-402f2e3c99c5/2014-01-30T10:52:30_2014-01-30T14:20:30']})


In [ ]:
import pandas as pd
df = pd.read_json(data["results"][0])
print(df.head(3))

                     co count  co stdev  co n_meas
2014-01-30 10:52:30    204.62     6.232         26
2014-01-30 10:53:30    200.78     5.934         26
2014-01-30 10:54:30    201.54     5.176         25
